In [1]:
import openpyxl
import glob
import os
from openpyxl import Workbook
import xlwings as xw

In [2]:
# 絞り込むエクセルファイルが保存されているパスを入力してください
folderpath = "/Users/Miyo/Desktop/ガスクロ　プログラム/0.textから変換済み"
# 保存したいエクセルファイルの保存先パスを入力してください
save_path = "/Users/Miyo/Desktop/ガスクロ　プログラム/1.絞り込み済み"



#ピークが始まるセル
peak_cell_start = 44


same_substance_col = 'E'
possible_col = 'C'


#どこまでのピークを取得しますか？
peak_range = 30



In [3]:
"""
1.同じ名前の化合物を削除し、確率を元に絞り込みます。
"""

files = sorted(glob.glob(folderpath + "/*.xlsx"))

num=1

for file in files:
    filename, ext = os.path.splitext(os.path.basename(file))
    wb = openpyxl.load_workbook(file)
    ws = wb.worksheets[0]
    #peakスタートの位置を揃える（０）
    if ws.cell(peak_range+13, 1).value != 'Spectrum#':
        for cell in ws.iter_rows(min_col=1, max_col=1):
            if cell[0].value == 'Spectrum#':
                spectal_cell = cell[0]
                between= peak_range + 13 - spectal_cell.row 
                ws.insert_rows(spectal_cell.row-2, between)
                break
    #peakごとにリスト分けする（１）
    
    peak_num=1
    peak_cell_s = peak_cell_start
    for i in range(1,peak_range+1):
        peak_list = []
        sublist =[]
        flg = False
        for peak in ws['A'+str(peak_cell_s) +':A'+ str(ws.max_row)]: 
            if peak[0].value == peak_num:
                    sublist.append(peak[0])
            if flg:
                break
            elif peak[0].value == peak_num+1:
                flg = True
        #欠損ファイルは途中で止める
        if sublist == []:
            break
    
        # 同じ物質の削除の用意をする（２）
        delete_list = []
        peak_s=sublist[0].row
        peak_e=sublist[-1].row
        peak_re=sublist[-1].row
        sec = peak_s + 1
        for onazi in range(peak_s, peak_e):
            for onazi2 in range(sec,peak_e + 1):
                if ws[same_substance_col + str(onazi)].value == ws[same_substance_col + str(onazi2)].value:   #あとで列設定
                    delete_list.append(ws[same_substance_col + str(onazi2)].row)
            sec += 1
    
        #削除（３）
        dlt_list=sorted(set((delete_list)))
        dlt_num = 0
        dlt_sum=0

        for dlt in dlt_list:
            ws.delete_rows(dlt - dlt_num)
            dlt_num += 1
            dlt_sum+=1

        delete_list.clear()
        sublist.clear()
        #削除したため、ピークリストの再取得
        peak_list = []
        sublist =[]
        flg = False
        for peak in ws['A'+str(peak_cell_s) +':A'+ str(ws.max_row)]: #peakのセル、あとで設定
            if peak[0].value == peak_num:
                sublist.append(peak[0])
            if flg:
                break
            elif peak[0].value == peak_num+1:
                flg = True
        peak_num+=1


        peak_s=sublist[0].row
        peak_e=sublist[-1].row
        
        # 確率の重複確認（４）
        head_list =[] #重複を確認するリストを用意
        head_list.clear()
        for fin_del_tuple in ws[possible_col+str(peak_s)+':'+ possible_col +str(peak_s+7)]:
            head_list.append(fin_del_tuple[0].value)
        
        
        if head_list[2] == head_list[6]:
            final_del = ws[possible_col+str(peak_s+7)].row
        elif head_list[2] == head_list[5]:
            final_del = ws[possible_col+str(peak_s+6)].row
        elif head_list[2] == head_list[4]:
            final_del = ws[possible_col+str(peak_s+5)].row
        elif head_list[2] == head_list[3]:
            final_del = ws[possible_col+str(peak_s+4)].row
        else:
            final_del = ws[possible_col+str(peak_s+3)].row

        #最終削除（５）
        dlt_num = 0
        for dlt in range(final_del, peak_e+1):
            ws.delete_rows(dlt - dlt_num)
            dlt_num += 1
            dlt_sum+=1
        peak_re -= dlt_sum
        peak_cell_s = peak_re
        peak_list.clear()
        sublist.clear()
        dlt_list.clear()
    ws.insert_cols(3, 2)
    ws.delete_rows(1)
    ws.title=filename
    wb.save(save_path + '/'+ filename + ".xlsx" )
        
        
        
        
    

wb.close()    
    
print('絞り込みが終了しました')
print('IFERROR関数を入力してください')

絞り込みが終了しました
IFERROR関数を入力してください


In [4]:
"""
２.IFERROR関数を入力します。
  ファイルは上書き保存されます。
  新規で保存したい場合はsave_pathを消去し、パス名を入力してください。
  注意！開いているエクセルファイルをすべて閉じます。もしエクセルファイルを開いている場合は閉じてください。
"""
IFError_load_path=save_path

files = sorted(glob.glob(IFError_load_path + "/*.xlsx"))


for fil in files:
    xb=xw.Book(fil)
    max_row = xw.sheets[0].range('A'+str(peak_cell_start)).end('down').last_cell.row
    for iferror in range(peak_cell_start-1, max_row+1):
        xb.sheets[0].range((iferror,3)).formula= "=IFERROR(VLOOKUP(A"+str(iferror)+",$A$9:$H$"+ str(8+peak_range) +",2,FALSE),"")"
    
    for iferror in range(peak_cell_start-1, max_row+1):
        xb.sheets[0].range((iferror,4)).formula = '=IFERROR(VLOOKUP(A'+str(iferror)+',$A$9:$H$'+ str(8+peak_range) +',8,FALSE),"")'
    
    xb.save()
    app=xw.apps.active
    app.quit()

print('終了しました')


終了しました


In [5]:
"""
３.一つのファイルにまとめます。
　この処理ではRt.timeでの整列は行なっていません。
　パス名を入力し、実行してください。
"""

# まとめたいエクセルファイルが保存されているパスを入力してください
folderpath = "/Users/Miyo/Desktop/ガスクロ　プログラム/1.絞り込み済み"
# 保存したいエクセルファイルの保存先パスを入力してください
save_path = "/Users/Miyo/Desktop/ガスクロ　プログラム/3.まとめ"

files = sorted(glob.glob(folderpath + "/*.xlsx"))

wb2 = Workbook()
new_ws = wb2.active

initial_y, initial_x, second_x =1, 5, 5
y_header=1
file_num = 0

header_flg=True

for file in files:
    filename, ext = os.path.splitext(os.path.basename(file))
    wb = openpyxl.load_workbook(file, data_only=True)
    ws = wb.worksheets[0]
    file_num +=1
    peak_num=1
    peak_cell_s = peak_cell_start-1
    for i in range(1,peak_range+1):
            peak_list = []
            sublist =[]
            flg = False
            for peak in ws['A'+str(peak_cell_s) +':A'+ str(ws.max_row)]: 
                if peak[0].value == peak_num:
                    sublist.append(peak[0])
                if flg:
                    break
                elif peak[0].value == peak_num+1:
                    flg = True
            peak_num+=1
            if sublist == []:
                break
            peak_s=sublist[0].row
            peak_e=sublist[-1].row
            peak_cell_s = peak_e
#各セルから貼り付け（六セルごとに
            
            y_s = initial_y
            x_s = second_x
            x_header=5
            new_ws.cell(x_s, y_s).value = ws.cell(peak_s, 1).value #spectol
            new_ws.cell(x_s, y_s+1).value = ws.cell(peak_s, 3).value#peak
            new_ws.cell(x_s, y_s+2).value = ws.cell(peak_s, 4).value #area

            new_ws.cell(x_header-1, y_s).value = ws.cell(peak_cell_start-1,1).value #spectol header
            new_ws.cell(x_header-1, y_s+1).value = 'Rt.time'
            new_ws.cell(x_header-1, y_s+2).value = '面積'
            new_ws.cell(x_header-1, y_header+3).value = ws.cell(peak_cell_start-2, 6).value #CAS
            new_ws.cell(x_header-1, y_header+4).value = ws.cell(peak_cell_start-2, 7).value#Name
            new_ws.cell(x_header-1, y_header+5).value = ws.cell(peak_cell_start-2, 5).value#SI

            new_ws.cell(x_header-4, y_header+2).value = filename
            
            for CAS in ws['F'+str(peak_s)+':'+'F'+str(peak_e)]:
                new_ws.cell(x_s,y_s +3).value = CAS[0].value
                x_s += 1

            x_s = initial_x
            for Name in ws['G'+str(peak_s)+':'+'G'+str(peak_e)]:
                new_ws.cell(x_s,y_s +4).value = Name[0].value
                x_s += 1

            x_s = initial_x
            for SI in ws['E'+str(peak_s)+':'+'E'+str(peak_e)]:
                new_ws.cell(x_s,y_s +5).value = SI[0].value
                x_s += 1



            second_x+= 9
            initial_x+=9
    second_x=5
    initial_x=5
    initial_y+=6 
    y_header+=6

wb2.save(save_path + '/'+ '比較用' + ".xlsx" )

print('終了しました')

          






終了しました
